In [1]:
ls

sample_data/


In [4]:
import os
os.chdir('/content/drive/My Drive/final_code/noisystudent-master') # root dictionary, the parent folder root of train and test folder
os.getcwd()

'/content/drive/My Drive/final_code/noisystudent-master'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [6]:
import tensorflow as tf
tf.__version__

'1.15.2'

In [0]:
import sys

sys.path[0]='/tensorflow-1.15.2/python3.6'
sys.path

In [0]:
from __future__ import print_function

from keras.datasets import mnist
import keras
import random
from sklearn.metrics import accuracy_score
import numpy as np

from ladder_net import get_ladder_network_fc

inp_size = 28*28 # size of mnist dataset 
n_classes = 10
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, inp_size).astype('float32')/255
x_test  = x_test.reshape(10000,  inp_size).astype('float32')/255

y_train = keras.utils.to_categorical(y_train, n_classes)
y_test  = keras.utils.to_categorical(y_test,  n_classes)

idxs_label = range(x_train.shape[0])
idxs_unlabel = range(x_train.shape[0])
random.seed(0)
idxs_label = np.random.choice(x_train.shape[0], 1000)
random.seed(1)
idxs_unlabel = np.random.choice(x_train.shape[0], 50000)
x_train_unlabeled = x_train[idxs_unlabel]
y_train_unlabeled = y_train[idxs_unlabel]
x_train_labeled   = x_train[idxs_label]
y_train_labeled   = y_train[idxs_label]

n_rep = x_train_unlabeled.shape[0] // x_train_labeled.shape[0]
x_train_labeled_rep = np.concatenate([x_train_labeled]*n_rep)
y_train_labeled_rep = np.concatenate([y_train_labeled]*n_rep)
 
model = get_ladder_network_fc(layer_sizes=[inp_size, 1000, 500, 250, 250, 250, n_classes])

# train the model for 3 epochs
for _ in range(3):
    model.fit([x_train_labeled_rep, x_train_unlabeled], y_train_labeled_rep, epochs=1)
    y_test_pr = model.test_model.predict(x_test, batch_size=100)
    print("Test accuracy : %f" % accuracy_score(y_test.argmax(-1), y_test_pr.argmax(-1)))

Epoch 1/1
15904/50000 [========>.....................] - ETA: 1:17 - loss: 17.4496 - acc: 0.8710 - den_loss: 16.7641